In [14]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
def get_title_from_index(index):
	return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
	return df[df.title == title]["index"].values[0]

In [17]:
##Step 1: Read CSV File
df = pd.read_csv("https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv")
print (df.columns)

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')


In [18]:
df['popularity']

0       150.437577
1       139.082615
2       107.376788
3       112.312950
4        43.926995
           ...    
4798     14.269792
4799      0.642552
4800      1.444476
4801      0.857008
4802      1.929883
Name: popularity, Length: 4803, dtype: float64

In [19]:
'Alien' in df['title'].values

True

In [20]:
##Step 2: Select Features
features = ['keywords','cast','genres','director']

In [21]:
df[(df["director"] == 'David Fincher')]

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
100,100,150000000,Fantasy Drama Thriller Mystery Romance,http://www.benjaminbutton.com/,4922,diary navy funeral tea travel,en,The Curious Case of Benjamin Button,"Tells the story of Benjamin Button, a man who ...",60.269279,...,166.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Life isn't measured in minutes, but in moments.",The Curious Case of Benjamin Button,7.3,3292,Cate Blanchett Brad Pitt Tilda Swinton Julia O...,"[{'name': 'Eric Roth', 'gender': 2, 'departmen...",David Fincher
354,354,90000000,Thriller Crime Mystery Drama,http://dragontattoo.com/,65754,rape journalist based on novel journalism hacker,en,The Girl with the Dragon Tattoo,This English-language adaptation of the Swedis...,47.651083,...,158.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Evil shall with evil be expelled.,The Girl with the Dragon Tattoo,7.2,2434,Daniel Craig Rooney Mara Christopher Plummer G...,"[{'name': 'Steven Zaillian', 'gender': 2, 'dep...",David Fincher
421,421,65000000,Crime Drama Mystery Thriller,NaN,1949,california san francisco killing journalist ne...,en,Zodiac,The true story of the investigation of 'The Zo...,51.970905,...,157.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's more than one way to lose your life to...,Zodiac,7.3,2023,Jake Gyllenhaal Robert Downey Jr. Mark Ruffalo...,"[{'name': 'Victor J. Zolfo', 'gender': 2, 'dep...",David Fincher
662,662,63000000,Drama,http://www.foxmovies.com/movies/fight-club,550,support group dual identity nihilism rage and ...,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,146.757391,...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Mischief. Mayhem. Soap.,Fight Club,8.3,9413,Edward Norton Brad Pitt Meat Loaf Jared Leto H...,"[{'name': 'Arnon Milchan', 'gender': 2, 'depar...",David Fincher
693,693,61000000,Mystery Thriller Drama,http://www.gonegirlmovie.com/,210577,based on novel marriage crisis disappearance c...,en,Gone Girl,With his wife's disappearance having become th...,143.041543,...,145.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You don't know what you've got 'til it's...,Gone Girl,7.9,5862,Ben Affleck Rosamund Pike Carrie Coon Neil Pat...,"[{'name': 'Arnon Milchan', 'gender': 2, 'depar...",David Fincher
838,838,50000000,Science Fiction Action Horror,NaN,8077,prison android spacecraft space marine impriso...,en,Alien³,After escaping with Newt and Hicks from the al...,45.856409,...,114.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The bitch is back.,Alien³,6.2,1633,Sigourney Weaver Charles S. Dutton Charles Dan...,"[{'name': 'Jim Morahan', 'gender': 2, 'departm...",David Fincher
946,946,50000000,Drama Thriller Mystery,NaN,2649,brother brother relationship birthday danger o...,en,The Game,"In honor of his birthday, San Francisco banker...",62.925175,...,129.0,"[{""iso_639_1"": ""cn"", ""name"": ""\u5e7f\u5dde\u8b...",Released,What do you get for the man who has everything?,The Game,7.5,1506,Michael Douglas Sean Penn Deborah Kara Unger J...,"[{'name': 'Steve Golin', 'gender': 2, 'departm...",David Fincher
1010,1010,48000000,Crime Drama Thriller,http://www.sonypictures.com/movies/panicroom/,4547,burglar bunker housebreaking safe money,en,Panic Room,Trapped in their New York brownstone's panic r...,53.043124,...,111.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Panic Room,6.5,1267,Jodie Foster Kristen Stewart Forest Whitaker D...,"[{'name': 'Howard Shore', 'gender': 2, 'depart...",David Fincher
1161,1161,40000000,Drama,http://www.thesocialnetwork-movie.com/,37799,hacker hacking creator frat party social network,en,The Social Network,"On a fall night in 2003, Harvard undergrad and...",69.278220,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You d

In [23]:
##Step 3: Create a column in DF which combines all selected features
for feature in features:
	df[feature] = df[feature].fillna('')

def combine_features(row):
	try:
		return row['keywords'] +" "+row['cast']+" "+row["genres"]+" "+row["director"]
	except:
		print ("Error:", row)	

df["combined_features"] = df.apply(combine_features,axis=1)

print (df["combined_features"].head())

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
Name: combined_features, dtype: object


In [24]:
df[['keywords','cast','genres','director','combined_features']]

,keywords,cast,genres,director,combined_features
0,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,Action Adventure Fantasy Science Fiction,James Cameron,culture clash future space war space colony so...
1,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Adventure Fantasy Action,Gore Verbinski,ocean drug abuse exotic island east india trad...
2,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Action Adventure Crime,Sam Mendes,spy based on novel secret agent sequel mi6 Dan...
3,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Action Crime Drama Thriller,Christopher Nolan,dc comics crime fighter terrorist secret ident...
4,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Action Adventure Science Fiction,Andrew Stanton,based on novel mars medallion space travel pri...
...,...,...,...,...,...
4798,united states\u2013mexico barrier legs arms pa...,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Action Crime Thriller,Robert Rodriguez,united states\u2013mexico barrier legs arms pa...
4799,,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Comedy Romance,Edward Burns,Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800,date love at first sight narration investigati...,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Comedy Drama Romance TV Movie,Scott Smith,date love at first sight narration investigati...
4801,,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,,Daniel Hsia,Daniel Henney Eliza Coupe Bill Paxton Alan Ru...


In [25]:
##Step 4: Use BERT to contexualise the 'combined_features'
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [26]:
text_preprocessed = bert_preprocess_model(df["combined_features"])
text_preprocessed.keys()

dict_keys(['input_word_ids', 'input_type_ids', 'input_mask'])

In [27]:
bert_model = hub.KerasLayer(encoder_url)

In [28]:
bert_results = bert_model(text_preprocessed)

ResourceExhaustedError: Exception encountered when calling layer "keras_layer_3" (type KerasLayer).

 OOM when allocating tensor with shape[4803,12,128,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node transformer/layer_0/self_attention/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_restored_function_body_105464]

Call arguments received:
  • inputs={'input_word_ids': 'tf.Tensor(shape=(4803, 128), dtype=int32)', 'input_type_ids': 'tf.Tensor(shape=(4803, 128), dtype=int32)', 'input_mask': 'tf.Tensor(shape=(4803, 128), dtype=int32)'}
  • training=None

In [ ]:
bert_results.keys()

In [ ]:
bert_results['encoder_outputs']

In [31]:
##Step 5: Compute the Cosine Similarity based on the count_matrix
cosine_sim = cosine_similarity(count_matrix) 
movie_user_likes = "Cars"

In [32]:
## Step 6: Get index of this movie from its title
movie_index = get_index_from_title(movie_user_likes)

similar_movies =  list(enumerate(cosine_sim[movie_index]))

In [33]:
## Step 7: Get a list of similar movies in descending order of similarity score
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

In [34]:
## Step 8: Print titles of first 50 movies
i=0
for element in sorted_similar_movies:
		print (get_title_from_index(element[0]))
		i=i+1
		if i>50:
			break

Cars
Cars 2
The Fast and the Furious: Tokyo Drift
2 Fast 2 Furious
Herbie Fully Loaded
Back to the Future Part II
The Final Destination
Days of Thunder
Back to the Future
Larry the Cable Guy: Health Inspector
The Adventures of Rocky & Bullwinkle
Bolt
Aliens in the Attic
Furious 7
Witless Protection
Meet the Deedles
Turbo
A Bug's Life
Up
The SpongeBob Movie: Sponge Out of Water
Death Race
Little Fockers
Are We There Yet?
Rugrats in Paris: The Movie
The Emperor's New Groove
Back to the Future Part III
The Ant Bully
Night at the Museum: Secret of the Tomb
American Graffiti
Penguins of Madagascar
Return to Never Land
Free Birds
Saving Mr. Banks
Toy Story
The Animal
My Big Fat Greek Wedding 2
The Road to El Dorado
Toy Story 2
Ice Age: Dawn of the Dinosaurs
Jimmy Neutron: Boy Genius
The Jungle Book 2
Princess Mononoke
Stuart Little 2
Wreck-It Ralph
Shrek
Crocodile Dundee II
Marmaduke
Love the Coopers
The Fast and the Furious
Night at the Museum: Battle of the Smithsonian
Kung Fu Panda 3
